In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import datetime
import os

import dotenv
import pandas as pd
import numpy as np
rng = np.random.default_rng()

from mtclient import MTClient
import mtglobals

In [70]:
# For testing the distribution
#unique, counts = np.unique(sample, return_counts=True)
#print(np.asarray((unique, counts)).T)

In [71]:
client = MTClient()

Loading environment variables from .env
Using LIVE MTurk requester API
Your account balance is 4667.77


### (a) Get all hits since `date_cutoff`

In [72]:
date_cutoff = datetime.datetime(2022, 11, 10, 0, 0, 0)

In [73]:
all_hits = client.download_all_hits(start_cutoff=date_cutoff)

p2:DslfcgjelMHQtwVJf3FAWC+yz2dDpLfyG3SYiZ5MTvfe7lIp+b6AACn/5boCCHI=
p2:PMuUKcWeNOjK2RaNs0B2V/YVvAISJRCTS8v+Kuhe4hx8tdpAocjUkWyCm7bX7K4=
Creation 2022-11-07 13:59:32-08:00 before start_cutoff
Saving downloaded HIT data to ../results_2stage/all_hit_data.pkl


In [74]:
# Filter to get just the stage-1 HITs
stage1_hits = [h for h in all_hits if "instantly unlock" in h['Title']]
[(h['Title'], h['HITId'], h['CreationTime']) for h in stage1_hits]

[('Quick 3-question survey about work [<15 seconds], instantly unlock 2nd-stage HIT with higher reward',
  '3TZDZ3Y0JT7UM6OA2VTTFBZ9X0G915',
  datetime.datetime(2022, 11, 20, 16, 23, 57, tzinfo=tzlocal())),
 ('Quick 3-question survey about work [<15 seconds], instantly unlock 2nd-stage HIT with higher reward',
  '3SD15I2WD3V3EQ0RRS40JUN45HQ36G',
  datetime.datetime(2022, 11, 17, 15, 35, 37, tzinfo=tzlocal()))]

In [75]:
# Important: put the HITIds for the stage-1 HIT whose submissions you want to process here
stage1_ids = ["3TZDZ3Y0JT7UM6OA2VTTFBZ9X0G915"]
    #["3L7SUC0TTVV9M1MMVLTY6SMJS1R0MQ"]
    #['3BPP3MA3TDLO79JV3GRYR49YMRTELU',
    #          '3MJ28H2Y1F9WHBTZ57ENFR7F42K5OK','3HEADTGN2QTGCI0U3QD4DK6JSDPRV5',
    #          '3MJ28H2Y1F9WHBTZ57ENFR7F42YO5H','31SIZS5W5AGO7A4DHRJU7X7BBK0QRY',]
              #'3HXK2V1N4LGGQ1TEZ5ZHCBV1T5AG2D','3J94SKDEKJQ5DE2AFY074XVG0N85DG']
              #'3IHWR4LC7EE5871Y5A8L7ELKUIC8II','3I01FDIL6N9DEHJNXFY96QS4GCF2D8']
s1_submissions = []
for cur_s1_id in stage1_ids:
    s1_submissions.extend(client.get_hit_submissions(cur_s1_id))

### (b) Get subset of HITs for all workers who submitted the stage 1 hit(s) specified above

In [76]:
#stage1_hits = [h for h in stage1_hits if h['HITId'] in stage1_ids]
#all_stage1_submitters = []
#for cur_hit in stage1_hits:
#    cur_hit_id = cur_hit['HITId']
#    cur_submissions = client.get_hit_submissions(cur_hit_id)
#    print(len(cur_submissions))
#    cur_submitters = [s['WorkerId'] for s in cur_submissions]
#    all_stage1_submitters.extend(cur_submitters)

In [77]:
#print(all_stage1_submitters, end='')
s1_submitters = [s1_sub['WorkerId'] for s1_sub in s1_submissions]

In [78]:
len(s1_submitters)

100

### (c) For each stage-1 submitter, create, launch, and record info for their custom stage-2 HIT

In [79]:
# Convert the stage-2 question to xml, if it hasn't been converted already
dotenv.load_dotenv(mtglobals.dotenv_fpath, override=True)
stage2_html_fpath = os.getenv("STAGE2_HTML_FPATH")
stage2_xml_fpath = mtglobals.gen_xml(stage2_html_fpath)
# Also load the survey link
survey_url = os.getenv("SURVEY_URL")

In [80]:
survey_url

'https://cumc.co1.qualtrics.com/jfe/form/SV_dbQxB81tAUIFnZs'

In [81]:
# Load the already-launched HITs, if the file exists, for quick checking
if os.path.isfile(mtglobals.stage2_launched_fpath):
    launched_df = pd.read_csv(mtglobals.stage2_launched_fpath)
else:
    launched_df = None
# Loop over workers who submitted stage 1
for worker_num, cur_worker_id in enumerate(s1_submitters):
    print(f"Processing worker #{worker_num}, {cur_worker_id}")
    launched_qual_name, launched_qual_num, launched_offer_amt = mtglobals.check_launched(launched_df, cur_worker_id)
    if launched_qual_num != -1:
        print(f"Stage 2 HIT for worker {cur_worker_id} already launched")
        continue
    # If we're here, this worker_id hasn't had a stage 2 HIT launched yet
    cur_qual_info = mtglobals.get_current_qual()
    cur_qual_name = cur_qual_info['qual_name']
    cur_qual_id = cur_qual_info['qual_id']
    last_qual_num = cur_qual_info['last_qual_num']
    last_offer = cur_qual_info['last_offer_amt']
    print(f"Last used qual num: {last_qual_num}")
    print(f"Last used offer amt: {last_offer}")
    # Generate random offer amount
    # New new: randomly sampled
    cur_offer_amt = mtglobals.draw_random_wage()
    # The qual num just increments until it reaches 100, the highest allowed, then drops back to 0.
    # (just for tracking purposes)
    cur_qual_num = (last_qual_num + 1) % 100
    # Generate audio_id from qual_num
    cur_audio_id = str(cur_qual_num).zfill(2)
    print(f"Processing worker {cur_worker_id}, offer {cur_offer_amt}, qual_num {cur_qual_num}")
    # Using the functions to create the HIT params and content
    # Generate the xml for the question
    stage2_question = mtglobals.gen_custom_hit(stage2_xml_fpath, survey_url, cur_worker_id,
                                               cur_offer_amt, cur_audio_id)
    # Assign the custom qualification num for this worker
    qual_response = client.assign_stage2_quals(cur_worker_id, cur_qual_name,
                                               cur_qual_id, cur_qual_num,
                                               cur_offer_amt)
    print("Qualification assignment response:")
    print(qual_response)
    # Generate the data structure which enforces the qualification restriction
    stage2_requirements = mtglobals.gen_qual_restriction(cur_qual_id, cur_qual_num)
    # And launch the HIT
    cur_hit_title = f'Custom workplace survey HIT for worker id {cur_worker_id}'
    cur_hit_description = f'Custom workplace survey HIT for worker id {cur_worker_id}, 30 questions, ~15mins to complete'
    cur_hit_keywords = 'survey,workplace,work'
    launch_response = client.launch_custom_hit(cur_worker_id, cur_offer_amt,
                                               cur_hit_title, cur_hit_description,
                                               cur_hit_keywords, stage2_question,
                                               stage2_requirements)
    print("Launch response:")
    print(launch_response['ResponseMetadata']['HTTPStatusCode'])
    launched_time = launch_response['HIT']['CreationTime']
    # The response includes several fields that will be helpful later
    cur_hit_type_id = launch_response['HIT']['HITTypeId']
    cur_hit_id = launch_response['HIT']['HITId']
    #print("Created HITTypeId: {}".format(hit_type_id))
    print(f"Created HITId {cur_hit_id} for worker {cur_worker_id}")
    custom_url = client.mturk_environment['preview_url'] + f"?groupId={cur_hit_type_id}"
    print(f"You can work the HIT here: {custom_url}")
    mtglobals.write_log(
        f"Created custom stage-2 HIT for worker {cur_worker_id}, offer {cur_offer_amt}, URL {custom_url}"
    )
    # And append the data for this iteration to the .csv of launched workers
    launched_fpath = mtglobals.add_posted_worker(cur_worker_id, cur_offer_amt, cur_qual_name, cur_qual_id,
                                       cur_qual_num, cur_hit_id, cur_hit_type_id, custom_url, launched_time)
    print(f"New launched HIT successfully added to {launched_fpath}")
    print("=====[ end loop iteration ]=====")


Processing worker #0, ASX9CJSEMH0UH
Stage 2 HIT for worker ASX9CJSEMH0UH already launched
Processing worker #1, A3TDA1NWHNQFE3
Stage 2 HIT for worker A3TDA1NWHNQFE3 already launched
Processing worker #2, A19GSU0ST2SIMW
Stage 2 HIT for worker A19GSU0ST2SIMW already launched
Processing worker #3, A33QI63NVR8CUO
Stage 2 HIT for worker A33QI63NVR8CUO already launched
Processing worker #4, ALCPF5NANBDSZ
Stage 2 HIT for worker ALCPF5NANBDSZ already launched
Processing worker #5, A3ENQKD0V9VHBF
Stage 2 HIT for worker A3ENQKD0V9VHBF already launched
Processing worker #6, A1TJA3HLWCJUSX
Stage 2 HIT for worker A1TJA3HLWCJUSX already launched
Processing worker #7, A23UXKTVERRXNK
Stage 2 HIT for worker A23UXKTVERRXNK already launched
Processing worker #8, A2TDK0LD62WZ8Y
Stage 2 HIT for worker A2TDK0LD62WZ8Y already launched
Processing worker #9, A1IL81UMEC9OJI
Stage 2 HIT for worker A1IL81UMEC9OJI already launched
Processing worker #10, A3AN5NG00BCMBP
Stage 2 HIT for worker A3AN5NG00BCMBP already 

### Now run 02b_NotifyStage2.ipynb